## Importing necessary libraries

In [7]:
import os
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import sklearn.cluster as cluster
from numpy import array, random, sum, unique
from pandas import DataFrame, read_csv

from ydata_synthetic.synthesizers import ModelParameters, TrainParameters
from ydata_synthetic.synthesizers.regular import WGAN_GP
from pandas_profiling import ProfileReport

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn import preprocessing
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler

## Loading the data

In [2]:
df = pd.read_csv('data/diabetes_binary_health_indicators_BRFSS2015.csv')

## Running pandas-profiling to explore the data

In [12]:
profile = ProfileReport(df, title="Pandas Profiling Report for Diabetes Dataset", explorative=True)

In [13]:
profile.to_widgets()

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

/opt/anaconda3/envs/ydata/lib/python3.8/site-packages/scipy/stats/_stats_py.py:4868: RuntimeWarning: overflow encountered in long_scalars
  (2 * xtie * ytie) / m + x0 * y0 / (9 * m * (size - 2)))


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

### Saving the report as a HTML file

In [6]:
profile.to_file(r"pandas_profiling_reports/Diabetes_Pandas_Profiling_Report.html")

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

### Dropping duplicate rows

In [10]:
df.drop_duplicates(inplace = True)

## Creating train test splits

In [11]:
#Creating a test split from 20% of the data for testing the model
X_train_df, X_test_df, y_train_df, y_test_df = train_test_split(df.drop(['Diabetes_binary'], axis = 1), df['Diabetes_binary'], test_size = 0.2, stratify = df['Diabetes_binary'], random_state = 0)

In [14]:
#Adding the labels to the X_train_df
X_train_df['Diabetes_binary'] = y_train_df

## Data Synthesizing

### Creating synthetic data for minority class

In [15]:
#WGAN with Gradient Penalty is chosen as the GAN architecture
model = WGAN_GP

In [17]:
#Choosing the categorical and numerical columns
num_cols = ['BMI', 'MentHlth', 'PhysHlth']
cat_cols = list(X_train_df.drop(num_cols, axis = 1).columns)

In [18]:
#Selecting the rows of minority class from the data
train_data = X_train_df.loc[ X_train_df['Diabetes_binary']==1 ].copy()

In [23]:
print(f"Dataset info: Number of rows - {train_data.shape[0]}") 
print(f"Number of columns - {train_data.shape[1]}")

Dataset info: Number of rows - 28078
Number of columns - 22


In [24]:
#Setting the parameters of the GAN model
noise_dim = 32
dim = 128
batch_size = 128

log_step = 100
epochs = 200+1
learning_rate = 5e-4
beta_1 = 0.5
beta_2 = 0.9
models_dir = './cache'

In [25]:
#Setting the model parameters and the training step parameters of the GAN model
gan_args = ModelParameters(batch_size=batch_size, lr=learning_rate, betas=(beta_1, beta_2),
                           noise_dim=noise_dim,layers_dim=dim)

train_args = TrainParameters(epochs=epochs, sample_interval=log_step)

In [26]:
#Initializing the GAN model
synthesizer = model(gan_args, n_critic = 10)

2022-07-26 18:52:25.222149: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set


In [27]:
#Training the GAN model
synthesizer.train(data = train_data, train_arguments = train_args, num_cols = num_cols, cat_cols = cat_cols)

2022-07-26 18:52:46.109420: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
  0%|▏                                          | 1/201 [00:17<59:09, 17.75s/it]

Epoch: 0 | disc_loss: -0.295232892036438 | gen_loss: 0.21595656871795654


  1%|▍                                          | 2/201 [00:27<42:27, 12.80s/it]

Epoch: 1 | disc_loss: -0.32885152101516724 | gen_loss: 0.24259492754936218


  1%|▋                                          | 3/201 [00:36<37:06, 11.25s/it]

...

Epoch: 198 | disc_loss: -0.35798999667167664 | gen_loss: 0.3430030643939972


100%|████████████████████████████████████████▊| 200/201 [40:42<00:11, 11.65s/it]

Epoch: 199 | disc_loss: -0.3540396988391876 | gen_loss: 0.34568312764167786


100%|█████████████████████████████████████████| 201/201 [40:53<00:00, 12.21s/it]

Epoch: 200 | disc_loss: -0.359432190656662 | gen_loss: 0.34782177209854126


#### Generating 100k rows of minority data

In [30]:
#Generating synthetic data of 100k examples of diabetic patients
minority_synth_data = synthesizer.sample(100000)

Synthetic data generation: 100%|██████████████| 782/782 [00:09<00:00, 78.40it/s]


In [31]:
minority_synth_data.shape

(100096, 22)

### Generating 100k rows of majority data

In [ ]:
#Selecting the rows of majority class from the data
train_data = X_train_df.loc[ X_train_df['Diabetes_binary']==0 ].copy()

#WGAN with Gradient Penalty is chosen as the GAN architecture
model = WGAN_GP

#Setting the parameters of the GAN model
noise_dim = 32
dim = 128
batch_size = 128

log_step = 100
epochs = 200+1
learning_rate = 5e-4
beta_1 = 0.5
beta_2 = 0.9
models_dir = './cache'

#Setting the model parameters and the training step parameters of the GAN model
gan_args = ModelParameters(batch_size=batch_size, lr=learning_rate, betas=(beta_1, beta_2),
                           noise_dim=noise_dim,layers_dim=dim)

train_args = TrainParameters(epochs=epochs, sample_interval=log_step)

#Initializing the GAN model
synthesizer = model(gan_args, n_critic = 10)

#Training the GAN model
synthesizer.train(data = train_data, train_arguments = train_args, num_cols = num_cols, cat_cols = cat_cols)

#Generating synthetic data of 100k examples of non-diabetic patients
majority_synth_data = synthesizer.sample(100000)